In [15]:
#采用多分类：softmax 
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import xgboost as xgb
from collections import Counter#统计数据
from sklearn.tree import DecisionTreeClassifier      # 回归决策树
#from sklearn.linear_model import LinearRegression   # 线性回归
from sklearn.ensemble import RandomForestClassifier,GradientBoostingClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split,StratifiedKFold#分层K折交叉验证
from sklearn.preprocessing import LabelEncoder#标签编码器
from sklearn.metrics import accuracy_score,classification_report
from sklearn.utils import class_weight

In [12]:
#数据处理
def dm01():
    data = pd.read_csv('./红酒品质分类.csv')
    data.info()
    #抽取特征数据和标签数据
    x = data.iloc[:,:-1]
    y = data.iloc[:,-1]-3
    #print(Counter(y))
    #切分
    x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=23,stratify=y)#划分均匀
    #保存
    data_train = pd.concat([x_train,y_train],axis = 1)
    data_test = pd.concat([x_test,y_test],axis = 1)
    data_train.to_csv('./train.csv')
    data_test.to_csv('./test.csv')
dm01()    

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1599 entries, 0 to 1598
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         1599 non-null   float64
 1   volatile acidity      1599 non-null   float64
 2   citric acid           1599 non-null   float64
 3   residual sugar        1599 non-null   float64
 4   chlorides             1599 non-null   float64
 5   free sulfur dioxide   1599 non-null   float64
 6   total sulfur dioxide  1599 non-null   float64
 7   density               1599 non-null   float64
 8   pH                    1599 non-null   float64
 9   sulphates             1599 non-null   float64
 10  alcohol               1599 non-null   float64
 11  quality               1599 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 150.0 KB


In [16]:
#训练模型
def dm02():
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('./test.csv')
    #模型划分
    x_train = train_data.iloc[:,:-1]
    y_train = train_data.iloc[:,-1]
    x_test = test_data.iloc[:,:-1]
    y_test = test_data.iloc[:,-1]
    #假如权重平衡 因为数据集是样本不均衡的
    class_weight.compute_sample_weight('balanced',y_train)#平衡
    #模型创建
    estimator = xgb.XGBClassifier(max_depth = 5,n_estimators = 100,learning_rate = 0.1,objective = 'multi:softmax',random_state = 23)
    #训练
    estimator.fit(x_train,y_train)
    #模型评估
    print(estimator.score(x_test,y_test))
    #模型保存
    joblib.dump(estimator,'./wine.pkl')
dm02()

0.7


In [19]:
#模型预测
def dm03():
    train_data = pd.read_csv('train.csv')
    test_data = pd.read_csv('./test.csv')
    #模型划分
    x_train = train_data.iloc[:,:-1]
    y_train = train_data.iloc[:,-1]
    x_test = test_data.iloc[:,:-1]
    y_test = test_data.iloc[:,-1]
    #加载模型
    estimator = joblib.load('./wine.pkl')
    #创建网格交叉验证  找最优参数组合
    param_dict = {'max_depth':[2,3,5,6,7],'n_estimators':[50,120,150],'learning_rate':[0.2,0.3,0.5,1,1.3]}
    #创建分层采样数据对象
    skf = StratifiedKFold(n_splits=5,shuffle=True,random_state=23)
    #创建网格对象
    estimator1 = GridSearchCV(estimator,param_dict ,cv = skf)
    #模型训练
    estimator1.fit(x_train,y_train)
    y_pred = estimator1.predict(x_test)
    print(estimator1.best_estimator_)
    print(estimator1.best_score_)
    print(estimator1.best_params_)
dm03()   

XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.3, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=120, n_jobs=None,
              num_parallel_tree=None, objective='multi:softmax', ...)
0.6724080882352942
{'learning_rate': 0.3, 'max_depth': 5, 'n_estimators': 120}
